In [1]:
import copy
import numpy as np

from numpy.random import shuffle
from numba import njit
import matplotlib.pyplot as plt

from time import time
from tqdm import tqdm

import funcs as f
import utils as utl

In [2]:
def show(G):
    
    tmp = 'X | '
    for v in V:
        tmp += ' ' + v + ' '
    
    print(tmp)
    print('')
    
    for row in range(len(V)):
        
        tmp = V[row] + ' | '
        for col in range(Vsize):
            
            if G[row][col] == np.inf:
                tmp +=  ' i '
            else:
                w = int(G[row][col])
                if w == 0:
                    tmp +=  " 0 "
                else:
                    if w < 10 : tmp +=  ' ' + str(w) + ' '
                    if 10 <= w < 100 : tmp +=  ' ' + str(w)

            
        print(tmp)

---

![title](testg2.png)

In [3]:
#     0    1    2    3
V = ["S", "a", "b", "E"]

Vsize = len(V)

In [4]:
g = np.zeros((Vsize,Vsize), dtype=np.float32)

In [5]:
# Regular (input) edges
# S
g[0][1] = 5
g[0][2] = 5

# a
g[1][3] = 5
g[1][2] = 5

# b 
g[2][3] = 5

In [6]:
print("Edges")
for v in range(g.shape[0]):
    for v_ in range(g.shape[0]):

        if g[v][v_] != 0:
            print(V[v], " -> ", V[v_]) 
            

print("Anti-edges")
for v in range(g.shape[0]):
    for v_ in range(g.shape[0]):          
            
        if g[v][v_] != 0 and g[v_][v] == 0:
            print(V[v_], " -> ", V[v]) 

Edges
S  ->  a
S  ->  b
a  ->  b
a  ->  E
b  ->  E
Anti-edges
a  ->  S
b  ->  S
b  ->  a
E  ->  a
E  ->  b


---

In [7]:
#@njit
def iteration(G, F, V):
    
    # BFS
    Prev = np.full(Vsize, -1) # Prev node
    Q = [0] # Queue
    cursor = 0 # instead of popping elements of of the Q

    while cursor < Vsize:
        v_curr = Q[cursor]
        for v_ind, capacity in enumerate( G[v_curr] ):
            if F[v_curr, v_ind] != capacity and Prev[v_ind] == -1 and v_ind != v_curr and v_ind != 0:
                Q.append(v_ind)
                Prev[v_ind] = v_curr

        cursor += 1
        if cursor >= len(Q) : break

            
            
    # path reconstruction
    v_curr = Vsize - 1
    path = [v_curr]
    while True:
        v_prev = Prev[v_curr]
        v_curr = v_prev
        if v_curr == -1 : break
        path.append(v_curr)

    path = path[::-1]
    
    
    
    # path bottleneck
    bottleneck = np.inf # float('inf') doesn't work with numba
    for i in range(len(path) - 1):
        v_curr = path[i]
        v_next = path[i+1]

        local_w = G[v_curr, v_next] - F_[v_curr, v_next]
        if local_w < bottleneck: 
            bottleneck = local_w
    if bottleneck == np.inf : return F, True
    
    print("path: ", path, " | bottleneck: ", bottleneck)
    
    
    # path flow update
    for i in range(len(path) - 1):
        v_curr = path[i]
        v_next = path[i+1]

        F[v_curr, v_next] += bottleneck
        F[v_next, v_curr] -= bottleneck
        
    
    #print('')
    #print('----- New iteration -----')
    #show_path(path, V)
    #print('Bottleneck is: ', bottleneck)
    
    return F, False

In [8]:
F_ = np.zeros((Vsize, Vsize)) # Current flow

In [13]:
while True:
        F_, end = iteration(g, F_, V)
        if end : break

In [14]:
show(F_)
print('MaxFlow is: ', sum(F_[:, -1]))

X |  S  a  b  E 

S |  0  5  5  0 
a |  -5  0  0  5 
b |  -5  0  0  5 
E |  0  -5  -5  0 
MaxFlow is:  10.0
